In [1]:
import pandas as pd
from sklearn.datasets import load_wine

wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

In this section we'll check for performance increase when apply two kinds of data transformation:

1. Apply a standard scaler to the data
2. TODO: Add a PCA transformation to the data
3. TODO: Evaluate the clusters. Aske mentioned something with variance.

In [2]:
# Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = {
    "scaled": pd.DataFrame(scaler.fit_transform(df), columns=df.columns),
    "raw": df}

data["raw"] = data["raw"].assign(target=wine.target)
data["scaled"] = data["scaled"].assign(target=wine.target)

In [3]:
from pomegranate import GeneralMixtureModel, LogNormalDistribution, GammaDistribution, NormalDistribution
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

distributions = {
    "lognormal": LogNormalDistribution,
    "gamma":GammaDistribution,
    "normal":NormalDistribution
    }

components = range(1,7)

sklearn_models = [KMeans,GaussianMixture]

num_iters = 25

results = {'distribution':[],
           'mutual_information':[],
           'datatype':[],
           'n_components':[]}

for _ in range(3):

    for datatype, df in data.items():

        features, labels = df.drop('target', axis=1, inplace=False), df['target']

        for component in tqdm(components):
            
            for iteration in range(num_iters):
                
                X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.2,stratify=labels)

                for name, distribution in distributions.items():
                    if datatype == 'scaled' and name in ['lognormal','gamma']:
                        score = 0
                    else:
                        #print(f'Running {name} on {datatype} with {component} components')
                        try:
                            model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train)
                            model.fit(X_train)
                            y_pred = model.predict(X_test)
                            score = adjusted_mutual_info_score(y_test, y_pred)               
                        except ZeroDivisionError:
                            score = 0
                        except ValueError:
                            score = 0
                        

                    results['distribution'].append(name)
                    results['mutual_information'].append(score)
                    results['datatype'].append(datatype)
                    results['n_components'].append(component)

                for model in sklearn_models:

                    #print(f'Running {model.__name__} on {datatype} with {component} components')
                    
                    MODEL = model()
                    if hasattr(MODEL,'n_components'):
                        MODEL.n_components = component
                    if hasattr(MODEL,'n_clusters'):
                        MODEL.n_clusters = component

                    if hasattr(MODEL,'predict'):
                        MODEL.fit(X_train)
                        y_pred = MODEL.predict(X_test)
                    else:
                        y_pred = MODEL.fit_predict(X_test)

                    score = adjusted_mutual_info_score(y_test, y_pred)

                    results['distribution'].append(MODEL.__class__.__name__)
                    results['mutual_information'].append(score)
                    results['datatype'].append(datatype)
                    results['n_components'].append(component)
            
results = pd.DataFrame(results)

 83%|████████▎ | 5/6 [00:11<00:02,  2.90s/it]/var/folders/ds/rtdqdzb153306j602rcl9s740000gn/T/ipykernel_38862/3131363679.py:44: RuntimeWarning: invalid value encountered in subtract
  model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train)
 83%|████████▎ | 5/6 [00:10<00:02,  2.77s/it]/var/folders/ds/rtdqdzb153306j602rcl9s740000gn/T/ipykernel_38862/3131363679.py:44: RuntimeWarning: invalid value encountered in subtract
  model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train)
/var/folders/ds/rtdqdzb153306j602rcl9s740000gn/T/ipykernel_38862/3131363679.py:44: RuntimeWarning: invalid value encountered in subtract
  model = GeneralMixtureModel.from_samples(distribution, n_components=component,X=X_train)
100%|██████████| 6/6 [00:15<00:00,  2.54s/it]


In [4]:
results.groupby(['distribution','datatype','n_components']).mean().sort_values('mutual_information',ascending=False)

mutual_information
distribution    datatype n_components                    
GaussianMixture scaled   3                       0.902378
KMeans          scaled   3                       0.882433
gamma           raw      3                       0.839564
lognormal       raw      3                       0.839549
gamma           raw      4                       0.836859
normal          raw      3                       0.835478
                scaled   3                       0.834715
lognormal       raw      4                       0.824862
GaussianMixture scaled   4                       0.813804
lognormal       raw      5                       0.803827
KMeans          scaled   4                       0.795989
gamma           raw      5                       0.786993
normal          raw      4                       0.772945
gamma           raw      6                       0.749079
KMeans          scaled   5                       0.739155
GaussianMixture scaled   5                       0.738186
normal          raw      5                       0.731236
                scaled   4                       0.730015
                raw      6                       0.719695
lognormal       raw      6                       0.708272
normal          scaled   5                       0.688239
KMeans          scaled   6                       0.687590
GaussianMixture scaled   6                       0.681517
lognormal       raw      2                       0.669465
gamma           raw      2                       0.614969
normal          scaled   6                       0.597312
GaussianMixture raw      2                       0.587672
                         3                       0.576294
                scaled   2                       0.538806
normal          raw      2                       0.537518
                scaled   2                       0.515111
KMeans          scaled   2                       0.478319
GaussianMixture raw      5                       0.454701
                         4                       0.445811
                         6                       0.428349
KMeans          raw      2                       0.425323
                         3                       0.416318
                         5                       0.395210
                         6                       0.383858
                         4                       0.375696
lognormal       scaled   4                       0.000000
normal          scaled   1                       0.000000
                raw      1                       0.000000
lognormal       scaled   6                       0.000000
                         5                       0.000000
GaussianMixture raw      1                       0.000000
lognormal       scaled   3                       0.000000
                         2                       0.000000
                         1                       0.000000
                raw      1                       0.000000
gamma           scaled   6                       0.000000
                         5                       0.000000
                         4                       0.000000
                         3                       0.000000
                         2                       0.000000
                raw      1                       0.000000
KMeans          scaled   1                       0.000000
                raw      1                       0.000000
GaussianMixture scaled   1                       0.000000
gamma           scaled   1                       0.000000